# Setup

In [ ]:
# Versão anterior para usar o Tsfresh
!pip install scipy==1.14.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 MB 26.4 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.14.0 which is incompatible.


In [ ]:
!pip install "dask[dataframe]"

INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 4.8 MB/s eta 0:00:00


In [ ]:
!pip install tsfresh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.5/176.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 41.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import json
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.feature_selection import RFECV
from sklearn.model_selection import cross_validate
from sklearn.base import clone
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.decomposition import PCA

# models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
import xgboost as xgb
import lightgbm as lgb

In [ ]:
from tsfresh.feature_extraction import feature_calculators

In [ ]:
import os

path = '/content/drive/MyDrive/pads/'
os.chdir(path)

print(f'Current working directory: {os.getcwd()}')

Current working directory: /content/drive/MyDrive/pads


# Definições de Função

In [ ]:
from scipy.fft import fft, fftfreq

def calculate_harmonic_features(signal, sampling_rate=100):

    n = len(signal)
    fft_values = fft(signal)
    fft_magnitudes = np.abs(fft_values[:n // 2])
    freqs = fftfreq(n, d=1 / sampling_rate)[:n // 2]

    # Frequência fundamental e harmônicas
    fundamental_idx = np.argmax(fft_magnitudes[1:]) + 1
    fundamental_freq = freqs[fundamental_idx]

    # Amplitude máxima da frequência fundamental (1) e das Harmônicas (2 e 3)
    Hamp1 = fft_magnitudes[fundamental_idx]
    Hamp2 = fft_magnitudes[2 * fundamental_idx] if 2 * fundamental_idx < len(fft_magnitudes) else 0
    Hamp3 = fft_magnitudes[3 * fundamental_idx] if 3 * fundamental_idx < len(fft_magnitudes) else 0

    # Distorções Harmônicas
    HD2 = Hamp2 / Hamp1 if Hamp1 != 0 else 0
    HD3 = Hamp3 / Hamp1 if Hamp1 != 0 else 0

    # Distorção Harmônica Total (THD)
    harmonic_indices = [k * fundamental_idx for k in range(2, 8) if k * fundamental_idx < len(fft_magnitudes)]
    THD = np.sqrt(sum((fft_magnitudes[idx] / Hamp1) ** 2 for idx in harmonic_indices)) if Hamp1 != 0 else 0

    return [Hamp1, Hamp2, Hamp3, fundamental_freq, HD2, HD3, THD]

def getFeaturesSelect(subjects, sampling_rate=100):
    features = []
    for id in subjects:
        subject_features = []
        try:
            x = np.fromfile(f"./preprocessed/movement/{id:03d}_ml.bin", dtype=np.float32).reshape((-1, 976))
        except FileNotFoundError:
            print(f"File for subject {id} not found.")
            continue

        for col in x:
            harmonic_features = calculate_harmonic_features(col, sampling_rate)
            subject_features.extend(harmonic_features)

        features.append(subject_features)
    return features


In [ ]:
def filter_dataframe(df, wrists, tasks, sensors, axes, features, remove=3):

    '''
    remove = 3 (default): Multiclassificação
    remove = 2: PD vs CO
    remove = 0: PD vs DD
    '''

    valid_columns = set(
        f"{task}_{wrist}_{sensor}_{axis}_{feature}"
        for task in tasks
        for wrist in wrists
        for sensor in sensors
        for axis in axes
        for feature in features
    )

    valid_columns.add('label')

    ##
    quest_columns = set(
        f"Q{num}" for num in range(1,31)
    )

    valid_columns.update(quest_columns)
    ##

    filtered_columns = [col for col in df.columns if col in valid_columns]
    filtered_df = df[filtered_columns]

    filtered_df = filtered_df[filtered_df['label'] != remove]

    return filtered_df


In [ ]:
def evaluate_scaled(X, y, clf, kf, n_runs=1):
    acc_list = []
    bal_acc_list = []
    f1_macro_list = []
    precision_list = []
    recall_list = []
    confusion_matrices = []

    for _ in range(n_runs):
        for fold, (train_index, test_index) in enumerate(kf.split(X, y)):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            # StandardScaler dentro de cada fold
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)

            clf.fit(X_train_scaled, y_train)
            y_pred = clf.predict(X_test_scaled)

            acc = accuracy_score(y_test, y_pred)
            bal_acc = balanced_accuracy_score(y_test, y_pred)
            f1_macro = f1_score(y_test, y_pred, average='macro')
            precision = precision_score(y_test, y_pred, average='macro')
            recall = recall_score(y_test, y_pred, average='macro')
            cm = confusion_matrix(y_test, y_pred, labels=np.unique(y))

            acc_list.append(acc)
            bal_acc_list.append(bal_acc)
            f1_macro_list.append(f1_macro)
            precision_list.append(precision)
            recall_list.append(recall)
            confusion_matrices.append(cm)

    # Aggregate results
    mean_acc = np.mean(acc_list)
    std_acc = np.std(acc_list)
    mean_bal_acc = np.mean(bal_acc_list)
    std_bal_acc = np.std(bal_acc_list)
    mean_f1_macro = np.mean(f1_macro_list)
    std_f1_macro = np.std(f1_macro_list)
    mean_precision = np.mean(precision_list)
    std_precision = np.std(precision_list)
    mean_recall = np.mean(recall_list)
    std_recall = np.std(recall_list)

    confusion_matrix_all = np.sum(confusion_matrices, axis=0)

    return {
        'mean_acc': mean_acc,
        'std_acc': std_acc,
        'mean_bal_acc': mean_bal_acc,
        'std_bal_acc': std_bal_acc,
        'mean_f1_macro': mean_f1_macro,
        'std_f1_macro': std_f1_macro,
        'mean_precision': mean_precision,
        'std_precision': std_precision,
        'mean_recall': mean_recall,
        'std_recall': std_recall,
        'confusion_matrix_all': confusion_matrix_all
    }

# Gerar Dataframes

In [ ]:
quest = pd.read_csv('preprocessed/quest_pivotted.csv')
quest.drop(columns=['Unnamed: 0', 'subject_id'], inplace=True)

In [ ]:
# Pivot the DataFrame to reshape it
pivot_quest = quest.pivot(index='subject_id', columns='link_id', values='answer')

# Rename the columns to Q1 - Q30
pivot_quest.columns = [f'Q{col}' for col in pivot_quest.columns]

# Reset the index to make subject_id a column again
pivot_quest.reset_index(inplace=True)

# Display the resulting DataFrame
pivot_quest

,subject_id,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,Q21,Q22,Q23,Q24,Q25,Q26,Q27,Q28,Q29,Q30
0,1,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2,True,True,False,False,True,False,False,False,True,...,True,True,True,False,True,False,True,False,True,False
2,3,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,4,False,True,False,True,False,False,False,True,True,...,True,True,True,False,False,True,True,True,False,False
4,5,True,True,True,False,False,False,False,True,False,...,False,True,True,True,True,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,465,True,False,False,False,False,False,False,True,True,...,False,False,False,False,True,False,False,False,False,False
465,466,False,False,False,False,True,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
466,467,False,False,False,False,False,False,False,True,True,...,False,False,True,False,False,False,False,False,False,False
467,468,False,False,False,False,False,False,False,True,True,...,True,False,False,False,True,True,True,True,False,False


In [ ]:
file_list = pd.read_csv('preprocessed/file_list.csv')

X = getFeaturesSelect(file_list['id'].tolist())
y = file_list['label'].tolist()

df = pd.DataFrame(X)
df['label'] = y
df

,0,1,2,3,4,5,6,7,8,9,...,915,916,917,918,919,920,921,922,923,label
0,0.488724,0.097562,0.122272,3.176230,0.199626,0.250186,0.365803,0.467863,0.049133,0.076501,...,0.049248,0.198489,10.140708,1.389934,2.704285,2.049180,0.137065,0.266676,0.347037,0
1,1.662973,0.455918,0.113083,4.918033,0.274158,0.068001,0.291793,5.929081,0.360778,0.145913,...,0.065195,0.179992,50.153477,8.602920,1.818516,1.844262,0.171532,0.036259,0.185804,2
2,0.410515,0.229886,0.267695,0.102459,0.559994,0.652095,1.092406,0.238537,0.050026,0.071233,...,0.052147,0.105677,45.636230,6.012286,1.271813,2.356557,0.131744,0.027868,0.146587,0
3,0.409187,0.096516,0.091445,2.971311,0.235874,0.223481,0.493524,0.524944,0.299135,0.041702,...,0.049420,0.116853,37.102676,0.408911,2.742984,2.254098,0.011021,0.073930,0.079775,1
4,2.173083,0.175685,0.035965,4.610656,0.080846,0.016550,0.087274,9.392246,1.008857,0.359460,...,0.032649,0.040392,89.447594,3.841126,2.115369,5.532787,0.042943,0.023649,0.053443,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,0.752110,0.056520,0.046542,4.713115,0.075149,0.061882,0.145073,0.810546,0.078293,0.117734,...,0.758658,1.692500,20.988125,5.746763,2.386721,2.561475,0.273810,0.113718,0.297712,1
465,0.351292,0.075613,0.074748,3.790984,0.215241,0.212779,0.349349,0.402443,0.245228,0.152391,...,0.150547,0.204041,98.044914,1.748795,5.942607,1.844262,0.017837,0.060611,0.071609,0
466,4.507583,1.006726,0.168069,5.840164,0.223341,0.037286,0.232901,8.323369,1.988431,1.145279,...,0.039898,0.088378,39.861816,9.996924,2.415452,6.454918,0.250789,0.060596,0.262521,1
467,0.490227,0.312460,0.139665,3.893443,0.637379,0.284899,0.782445,1.360717,0.149965,0.085229,...,0.009716,0.274419,61.983704,11.561347,3.720882,2.663934,0.186522,0.060030,0.197379,1


In [ ]:
tasks = ["Relaxed1", "Relaxed2", "RelaxedTask1", "RelaxedTask2", "StretchHold", "HoldWeight",
         "DrinkGlas", "CrossArms", "TouchNose", "Entrainment1", "Entrainment2"]

wrists = ["Left", "Right"]

sensors = ["Accelerometer", "Gyroscope"]

axes = ["X", "Y", "Z"]

features = ['Hamp1', 'Hamp2', 'Hamp3', 'fundamental_freq', 'HD2', 'HD3', 'THD']

In [ ]:
column_names = []
for task in tasks:
    for wrist in wrists:
        for sensor in sensors:
            for axis in axes:
                for feature in features:
                    column_names.append(f"{task}_{wrist}_{sensor}_{axis}_{feature}")

column_names.append("label")

In [ ]:
df.columns = column_names
df

,Relaxed1_Left_Accelerometer_X_Hamp1,Relaxed1_Left_Accelerometer_X_Hamp2,Relaxed1_Left_Accelerometer_X_Hamp3,Relaxed1_Left_Accelerometer_X_fundamental_freq,Relaxed1_Left_Accelerometer_X_HD2,Relaxed1_Left_Accelerometer_X_HD3,Relaxed1_Left_Accelerometer_X_THD,Relaxed1_Left_Accelerometer_Y_Hamp1,Relaxed1_Left_Accelerometer_Y_Hamp2,Relaxed1_Left_Accelerometer_Y_Hamp3,...,Entrainment2_Right_Gyroscope_Y_HD3,Entrainment2_Right_Gyroscope_Y_THD,Entrainment2_Right_Gyroscope_Z_Hamp1,Entrainment2_Right_Gyroscope_Z_Hamp2,Entrainment2_Right_Gyroscope_Z_Hamp3,Entrainment2_Right_Gyroscope_Z_fundamental_freq,Entrainment2_Right_Gyroscope_Z_HD2,Entrainment2_Right_Gyroscope_Z_HD3,Entrainment2_Right_Gyroscope_Z_THD,label
0,0.488724,0.097562,0.122272,3.176230,0.199626,0.250186,0.365803,0.467863,0.049133,0.076501,...,0.049248,0.198489,10.140708,1.389934,2.704285,2.049180,0.137065,0.266676,0.347037,0
1,1.662973,0.455918,0.113083,4.918033,0.274158,0.068001,0.291793,5.929081,0.360778,0.145913,...,0.065195,0.179992,50.153477,8.602920,1.818516,1.844262,0.171532,0.036259,0.185804,2
2,0.410515,0.229886,0.267695,0.102459,0.559994,0.652095,1.092406,0.238537,0.050026,0.071233,...,0.052147,0.105677,45.636230,6.012286,1.271813,2.356557,0.131744,0.027868,0.146587,0
3,0.409187,0.096516,0.091445,2.971311,0.235874,0.223481,0.493524,0.524944,0.299135,0.041702,...,0.049420,0.116853,37.102676,0.408911,2.742984,2.254098,0.011021,0.073930,0.079775,1
4,2.173083,0.175685,0.035965,4.610656,0.080846,0.016550,0.087274,9.392246,1.008857,0.359460,...,0.032649,0.040392,89.447594,3.841126,2.115369,5.532787,0.042943,0.023649,0.053443,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,0.752110,0.056520,0.046542,4.713115,0.075149,0.061882,0.145073,0.810546,0.078293,0.117734,...,0.758658,1.692500,20.988125,5.746763,2.386721,2.561475,0.273810,0.113718,0.297712,1
465,0.351292,0.075613,0.074748,3.790984,0.215241,0.212779,0.349349,0.402443,0.245228,0.152391,...,0.150547,0.204041,98.044914,1.748795,5.942607,1.844262,0.017837,0.060611,0.071609,0
466,4.507583,1.006726,0.168069,5.840164,0.223341,0.037286,0.232901,8.323369,1.988431,1.145279,...,0.039898,0.088378,39.861816,9.996924,2.415452,6.454918,0.250789,0.060596,0.262521,1
467,0.490227,0.312460,0.139665,3.893443,0.637379,0.284899,0.782445,1.360717,0.149965,0.085229,...,0.009716,0.274419,61.983704,11.561347,3.720882,2.663934,0.186522,0.060030,0.197379,1


In [ ]:
quest

,subject_id,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,Q21,Q22,Q23,Q24,Q25,Q26,Q27,Q28,Q29,Q30
0,1,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2,True,True,False,False,True,False,False,False,True,...,True,True,True,False,True,False,True,False,True,False
2,3,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,4,False,True,False,True,False,False,False,True,True,...,True,True,True,False,False,True,True,True,False,False
4,5,True,True,True,False,False,False,False,True,False,...,False,True,True,True,True,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,465,True,False,False,False,False,False,False,True,True,...,False,False,False,False,True,False,False,False,False,False
465,466,False,False,False,False,True,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
466,467,False,False,False,False,False,False,False,True,True,...,False,False,True,False,False,False,False,False,False,False
467,468,False,False,False,False,False,False,False,True,True,...,True,False,False,False,True,True,True,True,False,False


In [ ]:
df_full = pd.concat([df, quest], axis=1).drop(columns=['subject_id'])
df_full

,Relaxed1_Left_Accelerometer_X_Hamp1,Relaxed1_Left_Accelerometer_X_Hamp2,Relaxed1_Left_Accelerometer_X_Hamp3,Relaxed1_Left_Accelerometer_X_fundamental_freq,Relaxed1_Left_Accelerometer_X_HD2,Relaxed1_Left_Accelerometer_X_HD3,Relaxed1_Left_Accelerometer_X_THD,Relaxed1_Left_Accelerometer_Y_Hamp1,Relaxed1_Left_Accelerometer_Y_Hamp2,Relaxed1_Left_Accelerometer_Y_Hamp3,...,Q21,Q22,Q23,Q24,Q25,Q26,Q27,Q28,Q29,Q30
0,0.488724,0.097562,0.122272,3.176230,0.199626,0.250186,0.365803,0.467863,0.049133,0.076501,...,False,False,False,False,False,False,False,False,False,False
1,1.662973,0.455918,0.113083,4.918033,0.274158,0.068001,0.291793,5.929081,0.360778,0.145913,...,True,True,True,False,True,False,True,False,True,False
2,0.410515,0.229886,0.267695,0.102459,0.559994,0.652095,1.092406,0.238537,0.050026,0.071233,...,False,False,False,False,False,False,False,False,False,False
3,0.409187,0.096516,0.091445,2.971311,0.235874,0.223481,0.493524,0.524944,0.299135,0.041702,...,True,True,True,False,False,True,True,True,False,False
4,2.173083,0.175685,0.035965,4.610656,0.080846,0.016550,0.087274,9.392246,1.008857,0.359460,...,False,True,True,True,True,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,0.752110,0.056520,0.046542,4.713115,0.075149,0.061882,0.145073,0.810546,0.078293,0.117734,...,False,False,False,False,True,False,False,False,False,False
465,0.351292,0.075613,0.074748,3.790984,0.215241,0.212779,0.349349,0.402443,0.245228,0.152391,...,False,False,False,False,False,False,False,False,False,False
466,4.507583,1.006726,0.168069,5.840164,0.223341,0.037286,0.232901,8.323369,1.988431,1.145279,...,False,False,True,False,False,False,False,False,False,False
467,0.490227,0.312460,0.139665,3.893443,0.637379,0.284899,0.782445,1.360717,0.149965,0.085229,...,True,False,False,False,True,True,True,True,False,False


In [ ]:
df.to_csv('features.csv')
df_full.to_csv('features_full.csv')

# Experimentos

In [ ]:
models = {
    'SVM': SVC(),
    'KNN': KNeighborsClassifier(),
    'LDA': LinearDiscriminantAnalysis(),
    'Decision Tree': DecisionTreeClassifier(),
    'Naive Bayes': GaussianNB(),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Extra Trees': ExtraTreesClassifier(random_state=42),
    #'XGBoost': xgb.XGBClassifier(random_state=42),
    'LightGBM': lgb.LGBMClassifier(verbose=-1, random_state=42)
}

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# não recomendado, mas o output é dificil de ler sem suprimir warnings

## Acc

### PD vs CO

In [ ]:
X = filter_dataframe(df, wrists, tasks, ["Accelerometer"], axes, features, remove=2)
y = X['label']
X = X.drop('label', axis=1)

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=20)

In [ ]:
for name, model in models.items():
    results = evaluate_scaled(X, y, model, kf)
    print(f"{name}:")
    print(f"Acurácia média: {results['mean_acc']:.2%}, Desvio padrão: {results['std_acc']:.2%}")
    print(f"Acurácia balanceada média: {results['mean_bal_acc']:.2%}, Desvio padrão: {results['std_bal_acc']:.2%}")
    print(f"F-score médio: {results['mean_f1_macro']:.2%}, Desvio padrão: {results['std_f1_macro']:.2%}\n")
    print(f"Matriz de Confusão:\n{results['confusion_matrix_all']}\n")

SVM:
Acurácia média: 78.31%, Desvio padrão: 1.13%
Acurácia balanceada média: 51.25%, Desvio padrão: 2.50%
F-score médio: 46.10%, Desvio padrão: 4.68%

Matriz de Confusão:
[[  2  77]
 [  0 276]]

KNN:
Acurácia média: 76.90%, Desvio padrão: 3.63%
Acurácia balanceada média: 53.48%, Desvio padrão: 4.96%
F-score médio: 51.84%, Desvio padrão: 7.38%

Matriz de Confusão:
[[  9  70]
 [ 12 264]]

LDA:
Acurácia média: 69.58%, Desvio padrão: 3.40%
Acurácia balanceada média: 62.45%, Desvio padrão: 3.81%
F-score médio: 60.50%, Desvio padrão: 3.58%

Matriz de Confusão:
[[ 39  40]
 [ 68 208]]

Decision Tree:
Acurácia média: 72.39%, Desvio padrão: 6.46%
Acurácia balanceada média: 59.61%, Desvio padrão: 8.83%
F-score médio: 59.61%, Desvio padrão: 8.99%

Matriz de Confusão:
[[ 29  50]
 [ 48 228]]

Naive Bayes:
Acurácia média: 45.92%, Desvio padrão: 3.84%
Acurácia balanceada média: 62.52%, Desvio padrão: 2.92%
F-score médio: 45.76%, Desvio padrão: 3.72%

Matriz de Confusão:
[[ 73   6]
 [186  90]]

Random 

### PD vs DD

In [ ]:
X = filter_dataframe(df, wrists, tasks, ["Accelerometer"], axes, features, remove=0)
y = X['label'].replace(2,0)
X = X.drop('label', axis=1)

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=20)

In [ ]:
for name, model in models.items():
    results = evaluate_scaled(X, y, model, kf)
    print(f"{name}:")
    print(f"Acurácia média: {results['mean_acc']:.2%}, Desvio padrão: {results['std_acc']:.2%}")
    print(f"Acurácia balanceada média: {results['mean_bal_acc']:.2%}, Desvio padrão: {results['std_bal_acc']:.2%}")
    print(f"F-score médio: {results['mean_f1_macro']:.2%}, Desvio padrão: {results['std_f1_macro']:.2%}\n")
    print(f"Matriz de Confusão:\n{results['confusion_matrix_all']}\n")

SVM:
Acurácia média: 71.79%, Desvio padrão: 1.62%
Acurácia balanceada média: 53.31%, Desvio padrão: 1.91%
F-score médio: 49.21%, Desvio padrão: 2.72%

Matriz de Confusão:
[[ 10 104]
 [  6 270]]

KNN:
Acurácia média: 72.56%, Desvio padrão: 3.10%
Acurácia balanceada média: 60.58%, Desvio padrão: 2.36%
F-score médio: 60.98%, Desvio padrão: 2.84%

Matriz de Confusão:
[[ 36  78]
 [ 29 247]]

LDA:
Acurácia média: 59.74%, Desvio padrão: 6.15%
Acurácia balanceada média: 56.58%, Desvio padrão: 7.93%
F-score médio: 55.37%, Desvio padrão: 7.12%

Matriz de Confusão:
[[ 56  58]
 [ 99 177]]

Decision Tree:
Acurácia média: 65.13%, Desvio padrão: 2.74%
Acurácia balanceada média: 58.64%, Desvio padrão: 2.93%
F-score médio: 58.48%, Desvio padrão: 2.92%

Matriz de Confusão:
[[ 49  65]
 [ 71 205]]

Naive Bayes:
Acurácia média: 42.82%, Desvio padrão: 3.59%
Acurácia balanceada média: 56.80%, Desvio padrão: 2.77%
F-score médio: 42.15%, Desvio padrão: 3.97%

Matriz de Confusão:
[[103  11]
 [212  64]]

Random 

## Gyro

### PD vs CO

In [ ]:
X = filter_dataframe(df, wrists, tasks, ["Gyroscope"], axes, features, remove=2)
y = X['label']
X = X.drop('label', axis=1)

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=20)

In [ ]:
for name, model in models.items():
    results = evaluate_scaled(X, y, model, kf)
    print(f"{name}:")
    print(f"Acurácia média: {results['mean_acc']:.2%}, Desvio padrão: {results['std_acc']:.2%}")
    print(f"Acurácia balanceada média: {results['mean_bal_acc']:.2%}, Desvio padrão: {results['std_bal_acc']:.2%}")
    print(f"F-score médio: {results['mean_f1_macro']:.2%}, Desvio padrão: {results['std_f1_macro']:.2%}\n")
    print(f"Matriz de Confusão:\n{results['confusion_matrix_all']}\n")

SVM:
Acurácia média: 78.31%, Desvio padrão: 0.69%
Acurácia balanceada média: 51.25%, Desvio padrão: 1.53%
F-score médio: 46.23%, Desvio padrão: 2.98%

Matriz de Confusão:
[[  2  77]
 [  0 276]]

KNN:
Acurácia média: 78.87%, Desvio padrão: 1.99%
Acurácia balanceada média: 57.47%, Desvio padrão: 3.51%
F-score médio: 57.86%, Desvio padrão: 4.61%

Matriz de Confusão:
[[ 15  64]
 [ 11 265]]

LDA:
Acurácia média: 69.58%, Desvio padrão: 4.14%
Acurácia balanceada média: 61.41%, Desvio padrão: 7.22%
F-score médio: 59.91%, Desvio padrão: 6.15%

Matriz de Confusão:
[[ 37  42]
 [ 66 210]]

Decision Tree:
Acurácia média: 73.24%, Desvio padrão: 4.54%
Acurácia balanceada média: 60.30%, Desvio padrão: 6.04%
F-score médio: 60.00%, Desvio padrão: 5.93%

Matriz de Confusão:
[[ 29  50]
 [ 45 231]]

Naive Bayes:
Acurácia média: 48.45%, Desvio padrão: 4.23%
Acurácia balanceada média: 62.42%, Desvio padrão: 3.74%
F-score médio: 47.90%, Desvio padrão: 3.91%

Matriz de Confusão:
[[ 69  10]
 [173 103]]

Random 

### PD vs DD

In [ ]:
X = filter_dataframe(df, wrists, tasks, ["Gyroscope"], axes, features, remove=0)
y = X['label'].replace(2,0)
X = X.drop('label', axis=1)

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=20)

In [ ]:
for name, model in models.items():
    results = evaluate_scaled(X, y, model, kf)
    print(f"{name}:")
    print(f"Acurácia média: {results['mean_acc']:.2%}, Desvio padrão: {results['std_acc']:.2%}")
    print(f"Acurácia balanceada média: {results['mean_bal_acc']:.2%}, Desvio padrão: {results['std_bal_acc']:.2%}")
    print(f"F-score médio: {results['mean_f1_macro']:.2%}, Desvio padrão: {results['std_f1_macro']:.2%}\n")
    print(f"Matriz de Confusão:\n{results['confusion_matrix_all']}\n")

SVM:
Acurácia média: 73.59%, Desvio padrão: 2.08%
Acurácia balanceada média: 55.37%, Desvio padrão: 2.57%
F-score médio: 52.10%, Desvio padrão: 4.38%

Matriz de Confusão:
[[ 13 101]
 [  2 274]]

KNN:
Acurácia média: 66.92%, Desvio padrão: 5.09%
Acurácia balanceada média: 56.86%, Desvio padrão: 7.04%
F-score médio: 56.66%, Desvio padrão: 7.64%

Matriz de Confusão:
[[ 37  77]
 [ 52 224]]

LDA:
Acurácia média: 58.21%, Desvio padrão: 4.56%
Acurácia balanceada média: 54.53%, Desvio padrão: 7.29%
F-score médio: 53.33%, Desvio padrão: 6.17%

Matriz de Confusão:
[[ 52  62]
 [101 175]]

Decision Tree:
Acurácia média: 62.05%, Desvio padrão: 4.18%
Acurácia balanceada média: 57.77%, Desvio padrão: 4.91%
F-score médio: 56.98%, Desvio padrão: 4.58%

Matriz de Confusão:
[[ 54  60]
 [ 88 188]]

Naive Bayes:
Acurácia média: 41.79%, Desvio padrão: 4.02%
Acurácia balanceada média: 55.28%, Desvio padrão: 3.63%
F-score médio: 41.23%, Desvio padrão: 4.26%

Matriz de Confusão:
[[100  14]
 [213  63]]

Random 

## Acc + Gyro

### PD vs CO

In [ ]:
X = filter_dataframe(df, wrists, tasks, sensors, axes, features, remove=2)
y = X['label']
X = X.drop('label', axis=1)

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=20)

In [ ]:
for name, model in models.items():
    results = evaluate_scaled(X, y, model, kf)
    print(f"{name}:")
    print(f"Acurácia média: {results['mean_acc']:.2%}, Desvio padrão: {results['std_acc']:.2%}")
    print(f"Acurácia balanceada média: {results['mean_bal_acc']:.2%}, Desvio padrão: {results['std_bal_acc']:.2%}")
    print(f"F-score médio: {results['mean_f1_macro']:.2%}, Desvio padrão: {results['std_f1_macro']:.2%}\n")
    print(f"Matriz de Confusão:\n{results['confusion_matrix_all']}\n")

SVM:
Acurácia média: 78.87%, Desvio padrão: 1.26%
Acurácia balanceada média: 52.54%, Desvio padrão: 2.35%
F-score médio: 48.67%, Desvio padrão: 4.52%

Matriz de Confusão:
[[  4  75]
 [  0 276]]

KNN:
Acurácia média: 80.56%, Desvio padrão: 2.25%
Acurácia balanceada média: 58.59%, Desvio padrão: 3.44%
F-score médio: 59.28%, Desvio padrão: 4.92%

Matriz de Confusão:
[[ 15  64]
 [  5 271]]

LDA:
Acurácia média: 80.28%, Desvio padrão: 1.54%
Acurácia balanceada média: 69.67%, Desvio padrão: 2.51%
F-score médio: 70.23%, Desvio padrão: 1.46%

Matriz de Confusão:
[[ 40  39]
 [ 31 245]]

Decision Tree:
Acurácia média: 70.99%, Desvio padrão: 6.08%
Acurácia balanceada média: 60.41%, Desvio padrão: 6.49%
F-score médio: 59.76%, Desvio padrão: 6.91%

Matriz de Confusão:
[[ 33  46]
 [ 57 219]]

Naive Bayes:
Acurácia média: 46.76%, Desvio padrão: 2.58%
Acurácia balanceada média: 61.29%, Desvio padrão: 2.19%
F-score médio: 46.41%, Desvio padrão: 2.45%

Matriz de Confusão:
[[ 69  10]
 [179  97]]

Random 

### PD vs DD

In [ ]:
X = filter_dataframe(df, wrists, tasks, sensors, axes, features, remove=0)
y = X['label'].replace(2,0)
X = X.drop('label', axis=1)

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=20)

In [ ]:
for name, model in models.items():
    results = evaluate_scaled(X, y, model, kf)
    print(f"{name}:")
    print(f"Acurácia média: {results['mean_acc']:.2%}, Desvio padrão: {results['std_acc']:.2%}")
    print(f"Acurácia balanceada média: {results['mean_bal_acc']:.2%}, Desvio padrão: {results['std_bal_acc']:.2%}")
    print(f"F-score médio: {results['mean_f1_macro']:.2%}, Desvio padrão: {results['std_f1_macro']:.2%}\n")
    print(f"Matriz de Confusão:\n{results['confusion_matrix_all']}\n")

SVM:
Acurácia média: 73.33%, Desvio padrão: 1.50%
Acurácia balanceada média: 54.94%, Desvio padrão: 2.16%
F-score médio: 51.27%, Desvio padrão: 4.01%

Matriz de Confusão:
[[ 12 102]
 [  2 274]]

KNN:
Acurácia média: 70.26%, Desvio padrão: 4.01%
Acurácia balanceada média: 59.67%, Desvio padrão: 4.43%
F-score médio: 60.11%, Desvio padrão: 5.16%

Matriz de Confusão:
[[ 39  75]
 [ 41 235]]

LDA:
Acurácia média: 68.21%, Desvio padrão: 2.05%
Acurácia balanceada média: 59.57%, Desvio padrão: 2.59%
F-score médio: 59.72%, Desvio padrão: 2.49%

Matriz de Confusão:
[[ 44  70]
 [ 54 222]]

Decision Tree:
Acurácia média: 63.85%, Desvio padrão: 3.18%
Acurácia balanceada média: 56.43%, Desvio padrão: 2.35%
F-score médio: 56.21%, Desvio padrão: 2.53%

Matriz de Confusão:
[[ 44  70]
 [ 71 205]]

Naive Bayes:
Acurácia média: 42.31%, Desvio padrão: 4.13%
Acurácia balanceada média: 56.42%, Desvio padrão: 3.55%
F-score médio: 41.59%, Desvio padrão: 4.44%

Matriz de Confusão:
[[103  11]
 [214  62]]

Random 

## Acc + Quest

### PD vs CO

In [ ]:
X = filter_dataframe(df_full, wrists, tasks, ["Accelerometer"], axes, features, remove=2)
y = X['label']
X = X.drop('label', axis=1)

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=20)

In [ ]:
for name, model in models.items():
    results = evaluate_scaled(X, y, model, kf)
    print(f"{name}:")
    print(f"Acurácia média: {results['mean_acc']:.2%}, Desvio padrão: {results['std_acc']:.2%}")
    print(f"Acurácia balanceada média: {results['mean_bal_acc']:.2%}, Desvio padrão: {results['std_bal_acc']:.2%}")
    print(f"F-score médio: {results['mean_f1_macro']:.2%}, Desvio padrão: {results['std_f1_macro']:.2%}\n")
    print(f"Matriz de Confusão:\n{results['confusion_matrix_all']}\n")

SVM:
Acurácia média: 82.25%, Desvio padrão: 3.40%
Acurácia balanceada média: 62.47%, Desvio padrão: 6.30%
F-score médio: 64.34%, Desvio padrão: 8.59%

Matriz de Confusão:
[[ 21  58]
 [  5 271]]

KNN:
Acurácia média: 79.72%, Desvio padrão: 4.04%
Acurácia balanceada média: 63.50%, Desvio padrão: 4.93%
F-score médio: 65.21%, Desvio padrão: 6.05%

Matriz de Confusão:
[[ 27  52]
 [ 20 256]]

LDA:
Acurácia média: 76.06%, Desvio padrão: 8.40%
Acurácia balanceada média: 73.36%, Desvio padrão: 11.43%
F-score médio: 69.71%, Desvio padrão: 10.22%

Matriz de Confusão:
[[ 54  25]
 [ 60 216]]

Decision Tree:
Acurácia média: 74.65%, Desvio padrão: 4.45%
Acurácia balanceada média: 63.58%, Desvio padrão: 10.86%
F-score médio: 62.37%, Desvio padrão: 10.54%

Matriz de Confusão:
[[ 35  44]
 [ 46 230]]

Naive Bayes:
Acurácia média: 77.75%, Desvio padrão: 3.72%
Acurácia balanceada média: 82.54%, Desvio padrão: 2.20%
F-score médio: 74.23%, Desvio padrão: 3.43%

Matriz de Confusão:
[[ 72   7]
 [ 72 204]]

Ran

### PD vs DD

In [ ]:
X = filter_dataframe(df_full, wrists, tasks, ["Accelerometer"], axes, features, remove=0)
y = X['label'].replace(2,0)
X = X.drop('label', axis=1)

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=20)

In [ ]:
for name, model in models.items():
    results = evaluate_scaled(X, y, model, kf)
    print(f"{name}:")
    print(f"Acurácia média: {results['mean_acc']:.2%}, Desvio padrão: {results['std_acc']:.2%}")
    print(f"Acurácia balanceada média: {results['mean_bal_acc']:.2%}, Desvio padrão: {results['std_bal_acc']:.2%}")
    print(f"F-score médio: {results['mean_f1_macro']:.2%}, Desvio padrão: {results['std_f1_macro']:.2%}\n")
    print(f"Matriz de Confusão:\n{results['confusion_matrix_all']}\n")

SVM:
Acurácia média: 72.56%, Desvio padrão: 1.74%
Acurácia balanceada média: 54.88%, Desvio padrão: 1.70%
F-score médio: 52.11%, Desvio padrão: 2.20%

Matriz de Confusão:
[[ 14 100]
 [  7 269]]

KNN:
Acurácia média: 74.10%, Desvio padrão: 5.64%
Acurácia balanceada média: 64.42%, Desvio padrão: 5.55%
F-score médio: 65.42%, Desvio padrão: 6.30%

Matriz de Confusão:
[[ 47  67]
 [ 34 242]]

LDA:
Acurácia média: 61.03%, Desvio padrão: 4.77%
Acurácia balanceada média: 58.80%, Desvio padrão: 4.88%
F-score médio: 57.27%, Desvio padrão: 4.81%

Matriz de Confusão:
[[ 61  53]
 [ 99 177]]

Decision Tree:
Acurácia média: 67.69%, Desvio padrão: 4.47%
Acurácia balanceada média: 60.75%, Desvio padrão: 6.99%
F-score médio: 60.39%, Desvio padrão: 6.61%

Matriz de Confusão:
[[ 50  64]
 [ 62 214]]

Naive Bayes:
Acurácia média: 43.33%, Desvio padrão: 4.32%
Acurácia balanceada média: 57.42%, Desvio padrão: 3.08%
F-score médio: 42.64%, Desvio padrão: 4.81%

Matriz de Confusão:
[[104  10]
 [211  65]]

Random 

## Gyro + Quest

### PD vs CO

In [ ]:
X = filter_dataframe(df_full, wrists, tasks, ["Gyroscope"], axes, features, remove=2)
y = X['label']
X = X.drop('label', axis=1)

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=20)

In [ ]:
for name, model in models.items():
    results = evaluate_scaled(X, y, model, kf)
    print(f"{name}:")
    print(f"Acurácia média: {results['mean_acc']:.2%}, Desvio padrão: {results['std_acc']:.2%}")
    print(f"Acurácia balanceada média: {results['mean_bal_acc']:.2%}, Desvio padrão: {results['std_bal_acc']:.2%}")
    print(f"F-score médio: {results['mean_f1_macro']:.2%}, Desvio padrão: {results['std_f1_macro']:.2%}\n")
    print(f"Matriz de Confusão:\n{results['confusion_matrix_all']}\n")

SVM:
Acurácia média: 83.94%, Desvio padrão: 2.61%
Acurácia balanceada média: 64.93%, Desvio padrão: 6.09%
F-score médio: 67.44%, Desvio padrão: 8.06%

Matriz de Confusão:
[[ 24  55]
 [  2 274]]

KNN:
Acurácia média: 81.13%, Desvio padrão: 4.51%
Acurácia balanceada média: 67.04%, Desvio padrão: 5.81%
F-score médio: 68.79%, Desvio padrão: 6.69%

Matriz de Confusão:
[[ 33  46]
 [ 21 255]]

LDA:
Acurácia média: 73.24%, Desvio padrão: 8.07%
Acurácia balanceada média: 70.99%, Desvio padrão: 10.81%
F-score médio: 66.82%, Desvio padrão: 9.40%

Matriz de Confusão:
[[ 53  26]
 [ 69 207]]

Decision Tree:
Acurácia média: 77.75%, Desvio padrão: 2.58%
Acurácia balanceada média: 65.69%, Desvio padrão: 4.42%
F-score médio: 65.81%, Desvio padrão: 3.46%

Matriz de Confusão:
[[ 35  44]
 [ 35 241]]

Naive Bayes:
Acurácia média: 76.90%, Desvio padrão: 2.90%
Acurácia balanceada média: 80.71%, Desvio padrão: 3.28%
F-score médio: 73.01%, Desvio padrão: 2.68%

Matriz de Confusão:
[[ 69  10]
 [ 72 204]]

Random

### PD vs DD

In [ ]:
X = filter_dataframe(df_full, wrists, tasks, ["Gyroscope"], axes, features, remove=0)
y = X['label'].replace(2,0)
X = X.drop('label', axis=1)

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=20)

In [ ]:
for name, model in models.items():
    results = evaluate_scaled(X, y, model, kf)
    print(f"{name}:")
    print(f"Acurácia média: {results['mean_acc']:.2%}, Desvio padrão: {results['std_acc']:.2%}")
    print(f"Acurácia balanceada média: {results['mean_bal_acc']:.2%}, Desvio padrão: {results['std_bal_acc']:.2%}")
    print(f"F-score médio: {results['mean_f1_macro']:.2%}, Desvio padrão: {results['std_f1_macro']:.2%}\n")
    print(f"Matriz de Confusão:\n{results['confusion_matrix_all']}\n")

SVM:
Acurácia média: 73.85%, Desvio padrão: 2.24%
Acurácia balanceada média: 55.80%, Desvio padrão: 2.90%
F-score médio: 52.82%, Desvio padrão: 4.88%

Matriz de Confusão:
[[ 14 100]
 [  2 274]]

KNN:
Acurácia média: 67.18%, Desvio padrão: 2.64%
Acurácia balanceada média: 58.37%, Desvio padrão: 4.54%
F-score médio: 58.00%, Desvio padrão: 4.84%

Matriz de Confusão:
[[ 42  72]
 [ 56 220]]

LDA:
Acurácia média: 59.74%, Desvio padrão: 6.47%
Acurácia balanceada média: 55.38%, Desvio padrão: 7.12%
F-score médio: 54.66%, Desvio padrão: 6.70%

Matriz de Confusão:
[[ 51  63]
 [ 94 182]]

Decision Tree:
Acurácia média: 65.64%, Desvio padrão: 5.09%
Acurácia balanceada média: 60.33%, Desvio padrão: 5.54%
F-score médio: 59.86%, Desvio padrão: 5.41%

Matriz de Confusão:
[[ 54  60]
 [ 74 202]]

Naive Bayes:
Acurácia média: 43.08%, Desvio padrão: 5.59%
Acurácia balanceada média: 55.68%, Desvio padrão: 4.73%
F-score médio: 42.67%, Desvio padrão: 5.86%

Matriz de Confusão:
[[ 98  16]
 [206  70]]

Random 

## Acc + Gyro + Quest

### PD vs CO

In [ ]:
X = filter_dataframe(df_full, wrists, tasks, sensors, axes, features, remove=2)
y = X['label']
X = X.drop('label', axis=1)

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=20)

In [ ]:
for name, model in models.items():
    results = evaluate_scaled(X, y, model, kf)
    print(f"{name}:")
    print(f"Acurácia média: {results['mean_acc']:.2%}, Desvio padrão: {results['std_acc']:.2%}")
    print(f"Acurácia balanceada média: {results['mean_bal_acc']:.2%}, Desvio padrão: {results['std_bal_acc']:.2%}")
    print(f"F-score médio: {results['mean_f1_macro']:.2%}, Desvio padrão: {results['std_f1_macro']:.2%}\n")
    print(f"Matriz de Confusão:\n{results['confusion_matrix_all']}\n")

SVM:
Acurácia média: 80.85%, Desvio padrão: 1.13%
Acurácia balanceada média: 57.45%, Desvio padrão: 2.46%
F-score médio: 57.37%, Desvio padrão: 4.22%

Matriz de Confusão:
[[ 12  67]
 [  1 275]]

KNN:
Acurácia média: 81.97%, Desvio padrão: 2.87%
Acurácia balanceada média: 63.09%, Desvio padrão: 4.20%
F-score médio: 65.34%, Desvio padrão: 5.25%

Matriz de Confusão:
[[ 23  56]
 [  8 268]]

LDA:
Acurácia média: 86.48%, Desvio padrão: 1.91%
Acurácia balanceada média: 80.84%, Desvio padrão: 2.41%
F-score médio: 80.60%, Desvio padrão: 2.01%

Matriz de Confusão:
[[ 56  23]
 [ 25 251]]

Decision Tree:
Acurácia média: 73.24%, Desvio padrão: 3.09%
Acurácia balanceada média: 62.31%, Desvio padrão: 6.63%
F-score médio: 61.39%, Desvio padrão: 5.53%

Matriz de Confusão:
[[ 34  45]
 [ 50 226]]

Naive Bayes:
Acurácia média: 77.75%, Desvio padrão: 2.73%
Acurácia balanceada média: 81.65%, Desvio padrão: 1.82%
F-score médio: 73.94%, Desvio padrão: 2.40%

Matriz de Confusão:
[[ 70   9]
 [ 70 206]]

Random 

### PD vs DD

In [ ]:
X = filter_dataframe(df_full, wrists, tasks, sensors, axes, features, remove=0)
y = X['label'].replace(2,0)
X = X.drop('label', axis=1)

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=20)

In [ ]:
for name, model in models.items():
    results = evaluate_scaled(X, y, model, kf)
    print(f"{name}:")
    print(f"Acurácia média: {results['mean_acc']:.2%}, Desvio padrão: {results['std_acc']:.2%}")
    print(f"Acurácia balanceada média: {results['mean_bal_acc']:.2%}, Desvio padrão: {results['std_bal_acc']:.2%}")
    print(f"F-score médio: {results['mean_f1_macro']:.2%}, Desvio padrão: {results['std_f1_macro']:.2%}\n")
    print(f"Matriz de Confusão:\n{results['confusion_matrix_all']}\n")

SVM:
Acurácia média: 73.33%, Desvio padrão: 1.50%
Acurácia balanceada média: 54.94%, Desvio padrão: 2.16%
F-score médio: 51.27%, Desvio padrão: 4.01%

Matriz de Confusão:
[[ 12 102]
 [  2 274]]

KNN:
Acurácia média: 72.56%, Desvio padrão: 4.63%
Acurácia balanceada média: 62.58%, Desvio padrão: 5.69%
F-score médio: 63.28%, Desvio padrão: 6.25%

Matriz de Confusão:
[[ 44  70]
 [ 37 239]]

LDA:
Acurácia média: 69.23%, Desvio padrão: 3.97%
Acurácia balanceada média: 62.09%, Desvio padrão: 4.05%
F-score médio: 62.24%, Desvio padrão: 4.08%

Matriz de Confusão:
[[ 51  63]
 [ 57 219]]

Decision Tree:
Acurácia média: 64.10%, Desvio padrão: 2.29%
Acurácia balanceada média: 58.35%, Desvio padrão: 3.64%
F-score médio: 57.64%, Desvio padrão: 2.94%

Matriz de Confusão:
[[ 51  63]
 [ 77 199]]

Naive Bayes:
Acurácia média: 42.05%, Desvio padrão: 4.09%
Acurácia balanceada média: 55.99%, Desvio padrão: 3.37%
F-score médio: 41.36%, Desvio padrão: 4.41%

Matriz de Confusão:
[[102  12]
 [214  62]]

Random 